# N Body Simulation Code


### NBody with SYCL VIDEO - Presenter Dr. María Patoja
[![VIDEO](https://img.youtube.com/vi/rYe7guaZdTw/0.jpg)](https://www.youtube.com/watch?v=rYe7guaZdTw)

### SLIDES
[Explenation slides](https://docs.google.com/presentation/d/172OdlRp6vT-KKRXnS14x31WlCOFtjDzSFIRcJ3hkZ9A/edit?usp=sharing)

### NBody NVIDIA Simulations and visualization
[![VIDEO](https://img.youtube.com/vi/uhTuJZiAG64/0.jpg)](https://www.youtube.com/watch?v=uhTuJZiAG64)



In [1]:
%%writefile compute.cpp
#include <math.h>
#include <stdlib.h>
#include <omp.h>
#include <sys/time.h>
#include <sycl/sycl.hpp>
#include <oneapi/dpl/random>
#include <iostream>
#include <iomanip>

using namespace sycl;

// SOFTENING: This is used to prevent division by zero or extremely small distances during the force calculation.
#define SOFTENING 1e-9f

// Seed: This is used for random number generation.
#define seed 777

/* Body Structure: This defines a Body structure to represent each particle, including its position (x, y, z)
    and velocity (vx, vy, vz) in three dimensions.*/
typedef struct { float x, y, z, vx, vy, vz; } Body;

// Code to randomize Bodies on CPU
// The randomizeBodies function initializes the Body structures with random positions and velocities between -1.0 and 1.0. 
void randomizeBodies(Body *data, int n) {
    for (int i = 0; i < n; i++) {
        data[i].x = 2.0f * (rand() / (float)RAND_MAX) - 1.0f;
        data[i].y = 2.0f * (rand() / (float)RAND_MAX) - 1.0f;
        data[i].z = 2.0f * (rand() / (float)RAND_MAX) - 1.0f;
        data[i].vx = 2.0f * (rand() / (float)RAND_MAX) - 1.0f;
        data[i].vy = 2.0f * (rand() / (float)RAND_MAX) - 1.0f;
        data[i].vz = 2.0f * (rand() / (float)RAND_MAX) - 1.0f;      
    }
}

// Code that randomly initializes nbodies on GPU
/*
void randomizeBodies(Body *data, int n, queue &q) {
    q.submit([&](handler& h) { 
        h.parallel_for(range<1>(n), [=](item<1> idx) {
            auto offset = idx.get_linear_id();
            oneapi::dpl::minstd_rand engine(seed, offset);
            oneapi::dpl::uniform_real_distribution<float> distr(-1.0f, 1.0f);
            data[offset].x = distr(engine);
            data[offset].y = distr(engine);
            data[offset].z = distr(engine);
            data[offset].vx = distr(engine);
            data[offset].vy = distr(engine);
            data[offset].vz = distr(engine);
        });
    }).wait();
}
*/

void bodyForce(Body *data, float dt, int nBodies, queue &q){
    q.submit([&](handler& h) { //compute forces
            h.parallel_for(range<1>(nBodies), [=](id<1> i) {
                float Fx = 0.0f; float Fy = 0.0f; float Fz = 0.0f;
    
                for (int j = 0; j < nBodies; j++) {
                  float dx = data[j].x - data[i].x;
                  float dy = data[j].y - data[i].y;
                  float dz = data[j].z - data[i].z;
                  float distSqr = dx*dx + dy*dy + dz*dz + SOFTENING;
                  float invDist = 1.0f / sqrtf(distSqr);
                  float invDist3 = invDist * invDist * invDist;
    
                  Fx += dx * invDist3; Fy += dy * invDist3; Fz += dz * invDist3;
                }
    
                data[i].vx += dt*Fx; data[i].vy += dt*Fy; data[i].vz += dt*Fz;
            });
        });
}

int main(){
    // The main function initializes a SYCL queue 'q' and allocates an array of Body structures using malloc_shared.
    
    //# define queue which has default device associated for offload

    // If you want to run on GPU use:
    queue q;

    //if you want to run on CPU use:
    //queue q{cpu_selector_v};
    std::cout << "Device: " << q.get_device().get_info<info::device::name>() << "\n";

    // Amount of Bodies in the simulation
    int nBodies = 48000;

    // The main simulation loop runs for nIters iterations.
    const int nIters = 20;

    const float dt = 0.01f;
    Body *data = malloc_shared<Body>(nBodies, q);

    randomizeBodies(data, nBodies); //this can be also in a paralle_for and init on the GPU

    double start;
    double end;
    double totalTime = 0.0;
    double avgTime;

    for (int iter = 1; iter <= nIters; iter++) {
        //start timer here
        start = omp_get_wtime();

        // In each iteration, the bodyForce function calculates the gravitational force on each body due to every other body.
        // It updates the velocities of the bodies accordingly. It executes this calculation for each body in parallel, using the SYCL queue 'q'.
        bodyForce(data, dt, nBodies, q);

        // Positions are then updated in a separate parallel step.
        q.submit([&](handler& h) { //integrate positions
            h.parallel_for(range<1>(nBodies), [=](id<1> i) {
              data[i].x += data[i].vx*dt;
              data[i].y += data[i].vy*dt;
              data[i].z += data[i].vz*dt;
            });
        }).wait();
        // Wait() is to synchornize data before continuing

        //end timer here
        end = omp_get_wtime();
        // The time taken for each iteration is measured using omp_get_wtime().
        double tElapsed = end - start;

        if (iter > 1) { // The first iteration is considered a warm-up and is not included in the average timing.
          totalTime += tElapsed;
        }
        printf("Iteration %d: %.3f seconds\n", iter, tElapsed);
        avgTime = totalTime / (double)(nIters-1);
    }

    std::cout << "Nbodies: " << nBodies << "  Iterations: " << nIters << std::endl;
    std::cout << "Total Time Elapsed: " << std::fixed << std::setprecision(3) << totalTime << "\n";
    std::cout << "Average Time per Iteration: " << std::fixed << std::setprecision(4) << avgTime << std::endl;

    free(data, q);
    return 0;
}

Writing compute.cpp


In [4]:
%%writefile ./run-sycl.sh
#!/bin/bash 
source /opt/intel/oneapi/setvars.sh > /dev/null 2>&1
icpx -fsycl -fopemmp compute.cpp 

#export ONEAPI_DEVICE_SELECTOR=opencl:cpu 
#export ONEAPI_DEVICE_SELECTOR=opencl:gpu
#export ONEAPI_DEVICE_SELECTOR=level_zero:0


if [ $? -eq 0 ]; then ./a.out; fi

Overwriting ./run-sycl.sh


In [5]:
!chmod u+x ./run-sycl.sh &&./run-sycl.sh

icpx: error: unknown argument '-fopemmp'; did you mean '-fopenmp'?
